## Loading data from CSVs, cleaning and putting in mysql server

In [1]:
import sklearn
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

https://data.cityofnewyork.us/Housing-Development/NYC-Calendar-Sales-Archive-/uzf5-f8n2  <br>
https://data.cityofnewyork.us/City-Government/NYC-Citywide-Annualized-Calendar-Sales-Update/w2pb-icbu

## Combine all of the CSV files into one dataframe

In [2]:
#Taking all of the csv files from the links above create a single dataframe
folder_name = 'data'
data_dir = os.path.join(os.getcwd(),folder_name)
all_sales_df = pd.DataFrame() 

for file_name in os.listdir(data_dir):

    if file_name[0:4] in ['2011','2012','2013','2014','2015']:
        temp_frame = pd.read_excel(os.path.join(data_dir,file_name),header = 4)
    elif file_name[0:3] == 'NYC':
        temp_frame = pd.read_csv(os.path.join(data_dir,file_name),header = 0)
        temp_frame = temp_frame.rename(columns = {'TAX CLASS AS OF FINAL ROLL':'TAX CLASS AT PRESENT','BUILDING CLASS AS OF FINAL ROLL':'BUILDING CLASS AT PRESENT'})
        # temp_frame = temp_frame.rename(columns = {'EASE-MENT':'EASEMENT'})
        #Dropping columns that only appear in most recent data
        temp_frame = temp_frame.drop(columns = ["Latitude","Longitude","Community Board","Council District","Census Tract","BIN","BBL","NTA"])
    else:
        temp_frame = pd.read_excel(os.path.join(data_dir,file_name),header = 3)
    temp_frame.columns = temp_frame.columns.str.rstrip()
    if all_sales_df.empty == True:
        all_sales_df = temp_frame
    else:
        all_sales_df = all_sales_df.append(temp_frame)
#Dropping EASEMENT as there are only 13 in half a million records
#Dropping columns that aren't useful
#Using Apartment number would be nice to determine unit location in building if we have time
all_sales_df = all_sales_df.drop(columns = ["NEIGHBORHOOD","ADDRESS","TAX CLASS AT PRESENT","BUILDING CLASS AT PRESENT","LOT","EASE-MENT","APARTMENT NUMBER"])
#I need to reference these later so it cant have a space
all_sales_df = all_sales_df.rename(columns = {"SALE PRICE":"SALE_PRICE","RESIDENTIAL UNITS":"RESIDENTIAL_UNITS","LAND SQUARE FEET":"LAND_SQUARE_FEET","GROSS SQUARE FEET":"GROSS_SQUARE_FEET"})
int_cols = ['RESIDENTIAL_UNITS','COMMERCIAL UNITS','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT','SALE_PRICE']
all_sales_df.replace(',','',regex=True,inplace = True)

C:\Users\Will\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
all_sales_df.head()

,BOROUGH,BUILDING CLASS CATEGORY,BLOCK,ZIP CODE,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE,SALE DATE
0,2,01 ONE FAMILY HOMES,3035,10457.0,1,0,1,1287,1992,1899,1,A9,0,2009-12-07 00:00:00
1,2,01 ONE FAMILY HOMES,3046,10457.0,1,0,1,1964,1424,1899,1,A1,100,2009-04-28 00:00:00
2,2,01 ONE FAMILY HOMES,3046,10457.0,1,0,1,1964,1424,1899,1,A1,100,2009-04-13 00:00:00
3,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1768,1188,1901,1,A5,345752,2009-03-16 00:00:00
4,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1209,1048,1901,1,A1,0,2009-05-14 00:00:00


In [4]:
all_sales_df.columns

Index(['BOROUGH', 'BUILDING CLASS CATEGORY', 'BLOCK', 'ZIP CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE_PRICE', 'SALE DATE'],
      dtype='object')

In [5]:
for col in all_sales_df.columns:
    print(col, len(all_sales_df[col].unique()))

BOROUGH 5
BUILDING CLASS CATEGORY 145
BLOCK 13617
ZIP CODE 242
RESIDENTIAL_UNITS 740
COMMERCIAL UNITS 262
TOTAL UNITS 796
LAND_SQUARE_FEET 29023
GROSS_SQUARE_FEET 32030
YEAR BUILT 391
TAX CLASS AT TIME OF SALE 4
BUILDING CLASS AT TIME OF SALE 215
SALE_PRICE 106914
SALE DATE 6207


In [6]:
#Removing incomplete records, almost all of the dropped records had no sales price
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE != 0]
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE != -0]
all_sales_df = all_sales_df[all_sales_df.LAND_SQUARE_FEET != '-0']
all_sales_df = all_sales_df[all_sales_df.GROSS_SQUARE_FEET != '-0']
all_sales_df = all_sales_df[all_sales_df.LAND_SQUARE_FEET != '- 0']
all_sales_df = all_sales_df[all_sales_df.GROSS_SQUARE_FEET != '- 0']
all_sales_df = all_sales_df.dropna()


In [7]:
all_sales_df.shape

(1115050, 14)

In [8]:
all_sales_df = all_sales_df.astype({'RESIDENTIAL_UNITS':'int64'})
all_sales_df = all_sales_df.astype({'COMMERCIAL UNITS':'int64'})
all_sales_df = all_sales_df.astype({'TOTAL UNITS':'int64'})
all_sales_df = all_sales_df.astype({'LAND_SQUARE_FEET':'int64'})
all_sales_df = all_sales_df.astype({'GROSS_SQUARE_FEET':'int64'})
all_sales_df = all_sales_df.astype({'YEAR BUILT':'int64'})
all_sales_df = all_sales_df.astype({'SALE_PRICE':'int64'})


In [9]:
#Selecting properiteis with prices more than 100k and less than 10 mil as larger or smaller values tended to be outliers.
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE <= 10000000]
all_sales_df = all_sales_df[all_sales_df.SALE_PRICE >= 100000]

In [10]:
all_sales_df.shape

(932459, 14)

In [11]:
all_sales_df.head()

,BOROUGH,BUILDING CLASS CATEGORY,BLOCK,ZIP CODE,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE,SALE DATE
3,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1768,1188,1901,1,A5,345752,2009-03-16 00:00:00
6,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2352,1632,1899,1,B2,135000,2009-05-08 00:00:00
7,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2444,2125,1899,1,B2,325000,2009-03-30 00:00:00
10,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,344500,2009-11-20 00:00:00
11,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,316474,2009-05-04 00:00:00


In [12]:
all_sales_df['SALE DATE'] = pd.to_datetime(all_sales_df['SALE DATE'])

In [22]:
all_sales_df

,BOROUGH,BUILDING CLASS CATEGORY,BLOCK,ZIP CODE,RESIDENTIAL_UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE,SALE DATE
3,2,01 ONE FAMILY HOMES,3048,10457.0,1,0,1,1768,1188,1901,1,A5,345752,2009-03-16
6,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2352,1632,1899,1,B2,135000,2009-05-08
7,2,02 TWO FAMILY HOMES,3030,10457.0,2,0,2,2444,2125,1899,1,B2,325000,2009-03-30
10,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,344500,2009-11-20
11,2,02 TWO FAMILY HOMES,3036,10457.0,2,0,2,2000,2340,1998,1,B1,316474,2009-05-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9782,5,02 TWO FAMILY HOMES,7359,10309.0,2,0,2,5009,2796,2001,1,B2,730000,2006-03-27
9783,5,05 TAX CLASS 1 VACANT LAND,6913,10309.0,2,0,2,4000,2234,2006,1,V0,712775,2006-08-09
9794,5,05 TAX CLASS 1 VACANT LAND,7267,10309.0,2,0,2,5500,2688,2007,1,V0,365000,2006-06-28
9795,5,05 TAX CLASS 1 VACANT LAND,7267,10309.0,0,0,0,7920,0,0,1,V0,385000,2006-12-27


In [36]:
engine = create_engine('mysql+pymysql://db_editor:)k:3@3]nFN(}8!PR+!R&@6893.stephenshanko.com:3306/6893_project')

In [37]:
connection = pymysql.connect(host='6893.stephenshanko.com',
                         user='db_editor',
                         password=')k:3@3]nFN(}8!PR+!R&',
                         db='6893_project')   

In [38]:
cursor=connection.cursor()

In [39]:
all_sales_df.to_sql('Clean_Data',engine,if_exists='replace',index=False)

In [20]:
#If you want it as a local csv
# all_sales_df.to_csv('Cleaned_Sales_Data.csv')